In [5]:
# Import statements
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from keras import layers, models, regularizers

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import power_transform

import os
import winsound
from sklearn.preprocessing import LabelEncoder
import datetime as dt

os.chdir('../scripts')
from functions import impute_immediate_mean
os.chdir('../notebooks')


In [2]:
# Set up alarm for notification of model completion
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

## Read in Data

In [1855]:
# Read in data
df = pd.read_csv('../data/df_clean.csv', index_col=0, parse_dates=True)
X = df.drop(columns = 'price_tomorrow')
y = df.price_tomorrow

## Continuous

In [1857]:
X.drop(columns='diff', inplace=True)

continuous = X.select_dtypes(exclude='object').columns

# Get rid of negatives
time = dt.datetime(2021,3,23,22)
X.loc[time, 'dew_point_bilbao'] = impute_immediate_mean(X['dew_point_bilbao'], time)

# Add 0.0001 to everything
X[continuous] += .0001

# Box-Cox transformation
X[continuous] = power_transform(X[continuous], method='box-cox')

In [1858]:
df.drop(columns='diff', inplace=True)

continuous = df.select_dtypes(exclude='object').drop(columns='price_tomorrow').columns

# Get rid of negatives
time = dt.datetime(2021,3,23,22)
df.loc[time, 'dew_point_bilbao'] = impute_immediate_mean(df['dew_point_bilbao'], time)

# Add 0.0001 to everything
df[continuous] += .0001

# Box-Cox transformation
df[continuous] = power_transform(df[continuous], method='box-cox')

## Categoricals

In [1859]:
# Get Categorical columns
categorical = X.select_dtypes(include='object')

# Instationate LabelEncoder, fit and transform on wind_direction cols
wind_dir_coder = LabelEncoder()
wind_dir_coder.fit(X['wind_madrid'])
for col in categorical.filter(regex='wind').columns:
    X[col] = wind_dir_coder.transform(X[col])
    

# Stack condition columns into single col
stacked_conditions = categorical.filter(regex='condition').stack()

# Instantiate Label encoder, fit and transform on condition cols
condition_coder = LabelEncoder()
condition_coder.fit(stacked_conditions)
for col in categorical.filter(regex='condition').columns:
    X[col] = condition_coder.transform(X[col])

In [1860]:
# Get Categorical columns
categorical = df.select_dtypes(include='object')

# Instationate LabelEncoder, fit and transform on wind_direction cols
wind_dir_coder = LabelEncoder()
wind_dir_coder.fit(df['wind_madrid'])
for col in categorical.filter(regex='wind').columns:
    df[col] = wind_dir_coder.transform(df[col])
    

# Stack condition columns into single col
stacked_conditions = categorical.filter(regex='condition').stack()

# Instantiate Label encoder, fit and transform on condition cols
condition_coder = LabelEncoder()
condition_coder.fit(stacked_conditions)
for col in categorical.filter(regex='condition').columns:
    df[col] = condition_coder.transform(df[col])

## Split Data

In [2543]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='price_tomorrow'), df['price_tomorrow'], test_size=.3,
                                                    random_state=17)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=.5, random_state=17)

## Neural Network

In [2575]:
X_train.shape[0]
y_train.shape

(42705,)

In [2586]:
# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(239, activation='relu', input_shape=(X_train.shape[1],)))
nn.add(layers.Dense(162, activation='relu'))
nn.add(layers.Dense(1, activation='relu'))

# Loss Metric to optimize
metric = tf.keras.metrics.MeanAbsolutePercentageError(name='MAPE')

# Create early stopping point
callback = keras.callbacks.EarlyStopping(
    patience=10,
    monitor='val_'+metric.name,
    mode='min',
    restore_best_weights=True
)


# Compile the model
nn.compile(loss=tf.keras.metrics.mean_absolute_error, 
           optimizer=keras.optimizers.Adam(learning_rate=.0001),
           metrics=metric)

# Fit the model
history = nn.fit(x= X_train,
                 y=y_train,
                 batch_size=256,
                 epochs = 100,
                 callbacks=[callback],
                 validation_data=(X_val, y_val),
)

preds = nn.predict(X_train)
preds_flat = preds.flatten()
print('r2:',r2_score(y_train, preds_flat))
print('r2:',(np.corrcoef(y_train, preds_flat)**2)[0][1])


Epoch 1/100
167/167 [==============================] - 2s 9ms/step - loss: 40.2995 - MAPE: 80.5231 - val_loss: 29.1340 - val_MAPE: 109.6590
Epoch 2/100
167/167 [==============================] - 2s 9ms/step - loss: 27.1484 - MAPE: 77.7106 - val_loss: 25.3137 - val_MAPE: 101.3543
Epoch 3/100
167/167 [==============================] - 2s 9ms/step - loss: 24.6347 - MAPE: 66.8989 - val_loss: 23.5959 - val_MAPE: 84.9707
Epoch 4/100
167/167 [==============================] - 2s 9ms/step - loss: 22.8638 - MAPE: 57.1172 - val_loss: 21.9135 - val_MAPE: 73.2996
Epoch 5/100
167/167 [==============================] - 2s 9ms/step - loss: 20.8764 - MAPE: 49.5911 - val_loss: 19.9120 - val_MAPE: 58.6953
Epoch 6/100
167/167 [==============================] - 2s 9ms/step - loss: 18.6571 - MAPE: 39.5992 - val_loss: 17.7658 - val_MAPE: 44.3702
Epoch 7/100
167/167 [==============================] - 2s 9ms/step - loss: 16.5235 - MAPE: 33.1770 - val_loss: 15.7247 - val_MAPE: 28.3909
Epoch 8/100
167/167 [====

Epoch 60/100
167/167 [==============================] - 2s 10ms/step - loss: 6.8919 - MAPE: 18.4638 - val_loss: 7.3117 - val_MAPE: 19.4342
Epoch 61/100
167/167 [==============================] - 2s 10ms/step - loss: 6.8784 - MAPE: 18.5504 - val_loss: 7.3074 - val_MAPE: 19.5155
Epoch 62/100
167/167 [==============================] - 2s 10ms/step - loss: 6.8380 - MAPE: 18.6803 - val_loss: 7.2923 - val_MAPE: 19.2954
Epoch 63/100
167/167 [==============================] - 2s 10ms/step - loss: 6.8231 - MAPE: 18.6901 - val_loss: 7.2798 - val_MAPE: 19.2862
Epoch 64/100
167/167 [==============================] - 2s 10ms/step - loss: 6.8098 - MAPE: 19.0009 - val_loss: 7.2418 - val_MAPE: 19.2535
Epoch 65/100
167/167 [==============================] - 2s 11ms/step - loss: 6.7744 - MAPE: 19.2276 - val_loss: 7.2302 - val_MAPE: 19.2464
Epoch 66/100
167/167 [==============================] - 2s 11ms/step - loss: 6.7445 - MAPE: 19.4249 - val_loss: 7.1954 - val_MAPE: 19.2272
Epoch 67/100
167/167 [=====

## Neural Network - 24 Hour Prediction 
Two hidden layers
### Data Preparation

In [2591]:
def to_supervised(train, n_input, n_out=7, stride=1):
    # flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            X.append(data[in_start:in_end, :-1])
            y.append(data[in_end:out_end, -1])
        # move along one time step
        in_start += stride
    return np.array(X), np.array(y)

train = df.loc[:'2019'].drop(columns='price_tomorrow')
val = df.loc['2020'].drop(columns='price_tomorrow')

train['price_tomorrow'] = df.loc[:'2019', 'price_tomorrow']
val['price_tomorrow'] = df.loc['2020', 'price_tomorrow']

train = np.array(np.split(train, len(train)/24))
val = np.array(np.split(val, len(val)/24))
print(train.shape)
print(val.shape)


X_train, y_train = to_supervised(train, n_input=24, n_out=24, stride=24)
X_val, y_val = to_supervised(val, n_input=24, n_out=24, stride=24)
input_shape=(X_train.shape[1], X_train.shape[2])

(1825, 24, 63)
(366, 24, 63)


In [2500]:
# Instantiate model and build layers
bm = models.Sequential()
bm.add(layers.Dense(239, activation='relu', input_shape=input_shape))
bm.add(layers.Dense(162, activation='relu'))
bm.add(TimeDistributed(layers.Dense(1)))

Notes:

* <u>monitor_metric & metric (MAPE)</u>:  chosen because sMAPE not available. Chance to hard code sMAPE and implement?
* <u>patience (10)</u>: chosen to prevent reaching local minimum
* <u>loss function (mean_absolute_error)</u>: chosen because as the electricity prices have large spikes, the Euclidean norm would put too much importance on the spiky prices
* 

In [2529]:
# Loss Metric to optimize
metric = tf.keras.metrics.MeanAbsolutePercentageError(name='MAPE')

# Create early stopping point
callback = keras.callbacks.EarlyStopping(
    patience=10,
    monitor='val_'+metric.name,
    mode='min',
    restore_best_weights=True
)


# Compile the model
bm.compile(loss=tf.keras.metrics.mean_absolute_error, 
           optimizer=keras.optimizers.Adam(),
           metrics=metric)

# Fit the model
history = bm.fit(x= X_train,
                 y=y_train,
                 epochs = 100,
                 callbacks=[callback],
                 batch_size=32,
                 validation_data=(X_val, y_val),
)

preds = bm.predict(X_train)
preds_flat = preds.flatten()
y_flat = y.flatten()
print('r2:',r2_score(y_flat, preds_flat))
print('r2:',(np.corrcoef(y_flat, preds_flat)**2)[0][1])


Epoch 1/100
57/57 [==============================] - 1s 20ms/step - loss: 4.5302 - MAPE: 11.2072 - val_loss: 9.5527 - val_MAPE: 47.3506
Epoch 2/100
57/57 [==============================] - 1s 19ms/step - loss: 4.4713 - MAPE: 11.1238 - val_loss: 9.2161 - val_MAPE: 45.6000
Epoch 3/100
57/57 [==============================] - 1s 19ms/step - loss: 4.4765 - MAPE: 10.9687 - val_loss: 9.5385 - val_MAPE: 47.1624
Epoch 4/100
57/57 [==============================] - 1s 19ms/step - loss: 4.4459 - MAPE: 11.1432 - val_loss: 9.6587 - val_MAPE: 47.9454
Epoch 5/100
57/57 [==============================] - 1s 19ms/step - loss: 4.4293 - MAPE: 11.0799 - val_loss: 9.1387 - val_MAPE: 45.5755
Epoch 6/100
57/57 [==============================] - 1s 18ms/step - loss: 4.4502 - MAPE: 10.8989 - val_loss: 9.6634 - val_MAPE: 48.2124
Epoch 7/100
57/57 [==============================] - 1s 18ms/step - loss: 4.4127 - MAPE: 10.8707 - val_loss: 9.1851 - val_MAPE: 45.5395
Epoch 8/100
57/57 [=============================

In [2525]:
pd.DataFrame({'true':y_flat, 'preds':preds_flat})


,true,preds
0,66.82,63.248184
1,63.35,58.041203
2,58.79,57.722218
3,57.44,54.467926
4,55.29,55.256794
...,...,...
43771,46.16,47.173023
43772,44.02,42.715710
43773,45.60,43.714024
43774,42.90,40.947201


In [2602]:
# Instantiate model and build layers
bm_1 = models.Sequential()
bm_1.add(layers.Dense(239, activation='relu', input_shape=input_shape))
bm_1.add(layers.Dense(162, activation='relu'))
bm_1.add(TimeDistributed(layers.Dense(1)))

# Loss Metric to optimize
metric = tf.keras.metrics.MeanAbsolutePercentageError(name='MAPE')

# Create early stopping point
callback = keras.callbacks.EarlyStopping(
    patience=10,
    monitor='val_'+metric.name,
    mode='min',
    restore_best_weights=True
)


# Compile the model
bm_1.compile(loss=tf.keras.metrics.mean_absolute_error, 
           optimizer=keras.optimizers.Adam(learning_rate=.001),
           metrics=metric)

# Fit the model
history = bm_1.fit(x= X_train,
                 y=y_train,
                 epochs = 100,
                 callbacks=[callback],
                 batch_size=64,
                 validation_data=(X_val, y_val),
)

preds = bm_1.predict(X_train)
preds_flat = preds.flatten()
y_flat = y.flatten()
print('r2:',r2_score(y_flat, preds_flat))
print('r2:',(np.corrcoef(y_flat, preds_flat)**2)[0][1])


Epoch 1/100
29/29 [==============================] - 1s 32ms/step - loss: 28.4785 - MAPE: 61.2213 - val_loss: 15.8054 - val_MAPE: 76.7869
Epoch 2/100
29/29 [==============================] - 1s 26ms/step - loss: 13.5066 - MAPE: 38.1474 - val_loss: 15.4523 - val_MAPE: 73.5521
Epoch 3/100
29/29 [==============================] - 1s 26ms/step - loss: 11.8312 - MAPE: 33.6902 - val_loss: 12.9269 - val_MAPE: 62.9064
Epoch 4/100
29/29 [==============================] - 1s 27ms/step - loss: 10.7809 - MAPE: 30.0472 - val_loss: 11.6690 - val_MAPE: 57.5567
Epoch 5/100
29/29 [==============================] - 1s 27ms/step - loss: 9.9354 - MAPE: 27.6038 - val_loss: 11.3053 - val_MAPE: 55.7581
Epoch 6/100
29/29 [==============================] - 1s 27ms/step - loss: 9.3389 - MAPE: 25.6257 - val_loss: 11.7013 - val_MAPE: 57.8691
Epoch 7/100
29/29 [==============================] - 1s 27ms/step - loss: 8.8611 - MAPE: 24.3111 - val_loss: 9.8667 - val_MAPE: 48.8357
Epoch 8/100
29/29 [===================

array([67.906136, 63.718063, 52.723362, ..., 43.11868 , 48.786148,
       45.24533 ], dtype=float32)

In [2079]:
count

1825

In [2085]:
actual_preds.shape

(45625,)

In [2092]:
preds[48+24+24].flatten().shape

(24,)

In [1833]:
(np.corrcoef(array_y_train, preds)**2)[0][1]

0.09676594151905078

In [1763]:
results = pd.DataFrame({'True': array_y_train, 'preds': preds})
results.head(60)

,True,preds
0,64.02,46.181618
1,58.46,43.122044
2,54.70,29.287264
3,54.91,28.165249
4,53.07,33.292831
5,54.23,32.618267
6,58.22,32.518761
7,67.55,34.743282
8,70.33,35.144485
9,71.26,37.870037


In [1173]:
for index, batch in enumerate(train_set):
    if index==0:
        inputs, targets = batch
        print(inputs.shape)
        print(targets.shape)    
for index, batch in enumerate(train_set):
    if index==1824:
        inputs_end, targets_end = batch   

(24, 1, 62)
(24,)


In [1174]:
targets

<tf.Tensor: shape=(24,), dtype=float64, numpy=
array([64.02, 58.46, 54.7 , 54.91, 53.07, 54.23, 58.22, 67.55, 70.33,
       71.26, 75.86, 73.65, 74.19, 71.51, 71.04, 71.24, 70.64, 72.85,
       82.55, 83.33, 83.23, 79.06, 76.2 , 71.75])>

In [526]:
bm.load_weights('checkpoints/')
#bm.save('models/{}'.format(name))
r2_score(y_tr)

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/

In [102]:
keras.

array([[ 0.60542916,  1.13491958, -0.53602249, ..., -0.60937955,
         1.98402493,  4.        ],
       [ 0.62771746,  1.13467981, -0.36416936, ..., -0.60937955,
         1.98402493,  4.        ],
       [ 0.61657714,  1.13346918, -0.52947867, ..., -0.13194659,
         2.15110321,  4.        ],
       ...,
       [ 1.66451673, -0.84885601, -0.11902295, ..., -0.60937955,
         1.00834037,  4.        ],
       [ 1.60146487, -0.84885601, -0.20472313, ..., -0.13194659,
         0.85011052,  4.        ],
       [ 1.42163019, -0.84885601, -0.46096862, ..., -0.13194659,
         1.00834037,  4.        ]])

In [111]:
X.groupby(by=[X.index.year, X.index.month, X.index.day]).count().loc[X.groupby(by=[X.index.year, X.index.month, X.index.day]).count()['generation biomass']<24]

,,,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,...,wind_speeds_bilbao,pressures_bilbao,condition_bilbao,temp_valencia,dew_point_valencia,humidities_valencia,wind_valencia,wind_speeds_valencia,pressures_valencia,condition_valencia
2021,5,19,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [109]:
len(X.groupby(by=[X.index.year, X.index.month, X.index.day]).count())


2543